In [10]:
# 필요한거 import
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *

In [11]:
# 파일 불러오기
import pandas as pd
os.chdir('c:/temp')
tr = pd.read_json("train.json")
te = pd.read_json("val.json")

In [12]:
# 태그랑 언급횟수 딕셔너리로 저장
ret = []
for tag in tr.tags.tolist():
    ret += tag
from collections import Counter
r = dict(Counter(ret))

In [13]:
# 내림차순 정렬
r = sorted(r.items(), key=lambda x: -x[1])

In [14]:
# 상위권 태그 추출
top_tags = [x[0] for x in r[:1000]]

In [15]:
# train, val 안 songs,tags,ids 리스트로 변환
tr_songs = tr.songs.tolist()
te_songs = te.songs.tolist()
tr_tags = tr.tags.tolist()
te_tags = te.tags.tolist()
te_ids = te.id.tolist()

In [16]:
# tr 변수에 view, tag_to_idx 값 추가
from itertools import groupby
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

idx = 0
n_items = len(iid_to_idx)
for i, tags in enumerate(tr_tags):
    for tag in tags:
        if tag not in tag_to_idx:
            tag_to_idx[tag] = n_items + idx
            idx += 1
    tr[i].extend([tag_to_idx[x] for x in tags])
n_tags = len(tag_to_idx)
# n_items = len(iid_to_idx)
# n_tags = len(tag_to_idx)

In [17]:
# te변수에 ret 추가
from itertools import groupby
te = []

idx = 0
for i, l in enumerate(te_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)
idx = 0
for i, tags in enumerate(te_tags):
    ret = []
    for tag in tags:
        if tag not in tag_to_idx:
            continue
        ret.append(tag)
    te[i].extend([tag_to_idx[x] for x in ret])

In [19]:
# tr변수 내 데이터 섞기
tr = shuffle(tr)

In [20]:
# 데이터 딕셔너리화
idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}
idx_to_tag = {(x - n_items):y for(y,x) in tag_to_idx.items()}

In [49]:
# 희소행렬 생성
from scipy.sparse import csr_matrix

tr_csr = csr_matrix(tr.astype(float), (n_tags,n_items))
te_csr = csr_matrix(te.astype(float), (n_tags,n_items))

# tr_csr = rec_util.lil_to_csr(tr, (len(tr), n_tags + n_items))
# te_csr = rec_util.lil_to_csr(te, (len(te), n_tags + n_items))

ValueError: setting an array element with a sequence.

In [ ]:
# 희소행렬 행방향으로 붙이기
import scipy.sparse
r = scipy.sparse.vstack([te_csr, tr_csr])
r = csr_matrix(r)

In [ ]:
# als 모델링
als_model = ALS(factors=128, regularization=0.08)
als_model.fit(r.T * 15.0)

In [ ]:
# 적용
item_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
item_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

In [ ]:
item_model.item_factors = als_model.item_factors[:n_items]
tag_model.item_factors = als_model.item_factors[n_items:]

In [ ]:
# 학습된 추천 아이템, 태그 저장
item_ret = []
tag_ret = []
from tqdm.auto import tqdm
for u in tqdm(range(te_csr.shape[0])):
    item_rec = item_model.recommend(u, item_rec_csr, N=100)
    item_rec = [idx_to_iid[x[0]] for x in item_rec]
    tag_rec = tag_model.recommend(u, tag_rec_csr, N=100)
    tag_rec = [idx_to_tag[x[0]] for x in tag_rec if x[0] in idx_to_tag]
    item_ret.append(item_rec)
    tag_ret.append(tag_rec)

In [ ]:
tag_model.item_factors

In [ ]:
# id, songs, tags로 데이터 정렬
returnval = []
for _id, rec, tag_rec in zip(te_ids, item_ret, tag_ret):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [ ]:
# 제출용 파일 생성
import json
with open('ret.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))